In [1]:
!pip install kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kazanova/sentiment140

100% 80.9M/80.9M [00:02<00:00, 44.2MB/s]
100% 80.9M/80.9M [00:02<00:00, 33.5MB/s]


In [5]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'
with ZipFile(dataset,'r')as zip:
  zip.extractall()
  print('dataset exttracted')

dataset exttracted


In [7]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [13]:
df.shape

(1599999, 6)

In [14]:
df.head(3)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


In [19]:
column_names=['target','id','date','flag','user','text']
df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding = 'ISO-8859-1')

In [21]:
df.head(3)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [23]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [25]:
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [28]:
df.replace({'target':{4:1}}, inplace=True)

Stemming

In [30]:
port_stem=PorterStemmer()

In [32]:
def stemming(content):
  stemmed=re.sub('[^a-zA-Z]',' ',content)
  stemmed=stemmed.lower()
  stemmed=stemmed.split()
  stemmed=[port_stem.stem(word) for word in stemmed if not word in stopwords.words('english')]
  stemmed=' '.join(stemmed)
  return stemmed

In [33]:
df['content']=df['text'].apply(stemming)

In [34]:
df.head(3)

,target,id,date,flag,user,text,content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...


In [36]:
X=df['content'].values
Y=df['target'].values

In [53]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [54]:
#vectorizing
vectorizer=TfidfVectorizer()

In [55]:
X_train=vectorizer.fit_transform(X_train)

In [56]:
X_test=vectorizer.transform(X_test)

In [ ]:
print(X_train)
print(X_test)

In [46]:
#training the model
log_model=LogisticRegression(max_iter=1000)

In [58]:
log_model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [59]:
#model evaluation
X_train_predictions=log_model.predict(X_train)
train_acc=accuracy_score(Y_train,X_train_predictions)
print("training accuracy:",train_acc)

training accuracy: 0.81018984375


In [60]:
X_test_predictions=log_model.predict(X_test)
test_acc=accuracy_score(Y_test,X_test_predictions)
print('test accuracy:',test_acc)

test accuracy: 0.7780375


In [61]:
#saving the model
import pickle

In [62]:
filename='trained_model.sav'
pickle.dump(log_model, open(filename, 'wb'))

In [63]:
# using the model for further predictions
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [68]:
X_new=X_test[3]
print(Y_test[3])
predictions=loaded_model.predict(X_new)
print(predictions)
if(predictions[0]==0):
  print('Negetive tweet')
else:
  print('Positive Tweet')

0
[0]
Negetive tweet
